In [1]:
import requests
import json 
from tqdm import tqdm
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup 

In [2]:
# 주별
weeks = list(range(1,53))
# 연도
years = list(range(2018,2023))

In [3]:
# url 리스트 만들기
url = "http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime={}&hitYear={}&termGbn=week"

In [4]:
#url 리스트 만들기 
url_lists = []

for year in years:
    n1 = year
    for week in weeks:
        n2 = week
        url_lists.append(url.format(n2,n1))

In [5]:
url_lists

['http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=1&hitYear=2018&termGbn=week',
 'http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=2&hitYear=2018&termGbn=week',
 'http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=3&hitYear=2018&termGbn=week',
 'http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=4&hitYear=2018&termGbn=week',
 'http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=5&hitYear=2018&termGbn=week',
 'http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=6&hitYear=2018&termGbn=week',
 'http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=7&hitYear=2018&termGbn=week',
 'http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=8&hitYear=2018&termGbn=week',
 'http:/

In [6]:
#데이터 프레임 생성
kpop = {
    "name": [],
    "gaon": [],
    "week" : [],
}

In [7]:
for url in tqdm(url_lists):        
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')

    name = soup.select("div.chart > table > tr > td.subject > p:nth-of-type(1)")
    singer = soup.select("div.chart > table > tr > td.subject > p.singer")
    gaon = soup.select("div.chart > table > tr > td.count > p")

    for item2 in singer:
        kpop['name'].append(item2.text.replace('\n', '').replace('\t', '').replace('  ', ''))     
        
    for item3 in gaon:
        kpop['gaon'].append(item3.text.replace('\n', '').replace('\t', '').replace('  ', '')) 
        #kpop.append({'gaon': item3.text.replace('\n', '').replace('\t', '').replace('  ', '') })
        kpop['week'].append(url)
        
    


100%|█████████████████████████████████████████| 260/260 [15:52<00:00,  3.66s/it]


In [28]:
df = pd.DataFrame(kpop)
df.head()

,name,gaon,week
0,박효신|겨울소리,"57,579,742",http://gaonchart.co.kr/main/section/chart/onli...
1,DEAN|instagram,"49,766,243",http://gaonchart.co.kr/main/section/chart/onli...
2,문문 (MoonMoon)|LIFE IS BEAUTY FULL,"38,192,912",http://gaonchart.co.kr/main/section/chart/onli...
3,TWICE|Merry & Happy,"34,493,933",http://gaonchart.co.kr/main/section/chart/onli...
4,장덕철|그날처럼,"33,943,697",http://gaonchart.co.kr/main/section/chart/onli...


In [29]:
# 가수랑 곡명 구분
df['singer']= df['name'].apply(lambda x: x.split('|')[0] if pd.notnull(x) else x)
df['song']= df['name'].apply(lambda x: x.split('|')[1] if pd.notnull(x) else x)

# name 행 드롭
df = df.drop('name', axis=1)

df.head()

,gaon,week,singer,song
0,"57,579,742",http://gaonchart.co.kr/main/section/chart/onli...,박효신,겨울소리
1,"49,766,243",http://gaonchart.co.kr/main/section/chart/onli...,DEAN,instagram
2,"38,192,912",http://gaonchart.co.kr/main/section/chart/onli...,문문 (MoonMoon),LIFE IS BEAUTY FULL
3,"34,493,933",http://gaonchart.co.kr/main/section/chart/onli...,TWICE,Merry & Happy
4,"33,943,697",http://gaonchart.co.kr/main/section/chart/onli...,장덕철,그날처럼


In [30]:
df['year'] = df['week'].str.split('(\d+)', expand=True)[3]
df.head()

,gaon,week,singer,song,year
0,"57,579,742",http://gaonchart.co.kr/main/section/chart/onli...,박효신,겨울소리,2018
1,"49,766,243",http://gaonchart.co.kr/main/section/chart/onli...,DEAN,instagram,2018
2,"38,192,912",http://gaonchart.co.kr/main/section/chart/onli...,문문 (MoonMoon),LIFE IS BEAUTY FULL,2018
3,"34,493,933",http://gaonchart.co.kr/main/section/chart/onli...,TWICE,Merry & Happy,2018
4,"33,943,697",http://gaonchart.co.kr/main/section/chart/onli...,장덕철,그날처럼,2018


In [31]:
df['week'] = df['week'].str.split('(\d+)', expand=True)[1]
df.head()

,gaon,week,singer,song,year
0,"57,579,742",1,박효신,겨울소리,2018
1,"49,766,243",1,DEAN,instagram,2018
2,"38,192,912",1,문문 (MoonMoon),LIFE IS BEAUTY FULL,2018
3,"34,493,933",1,TWICE,Merry & Happy,2018
4,"33,943,697",1,장덕철,그날처럼,2018


In [32]:
# 파일 내보내기
df.to_csv("gaon-crawling.csv", index =False)